# N puzzle

Specifications of the problem : [https://en.wikipedia.org/wiki/15_puzzle](https://en.wikipedia.org/wiki/15_puzzle)

In [39]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from icecream import ic
import numpy as np

In [40]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['src', 'dst'])

In [41]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.src], new_state[action.dst] = new_state[action.dst], new_state[action.src]
    return new_state

In [42]:
RANDOMIZE_STEPS = 100_000
SOLUTION = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

state = SOLUTION.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

ic(state)
ic(SOLUTION)

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 209287.83it/s]
ic| state: array([[2, 8, 5],
                  [4, 0, 7],
                  [6, 3, 1]])
ic| SOLUTION: array([[1, 2, 3],
                     [4, 5, 6],
                     [7, 8, 0]])


array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])

In [43]:
def n_puzzle(state):
    frontier = list()
    while state is not None and not np.array_equal(SOLUTION, state):
        for action in available_actions(state):
            new_state = do_action(state, action)
            frontier.append(new_state)
        state = frontier.pop()
    return state

In [44]:
# final_state = n_puzzle(state)
# ic(final_state)

In [48]:
def n_puzzle_with_explored(state):
    """
    Résolution d'un puzzle en évitant les états redondants.
    Utilise une recherche en profondeur (DFS) avec un ensemble pour mémoriser les états visités.
    """
    frontier = [state]  # La pile des états à explorer
    explored = set()    # Ensemble des états déjà explorés

    while frontier:
        # Récupère le prochain état à explorer
        current_state = frontier.pop()

        # Vérifie si la solution est atteinte
        if np.array_equal(SOLUTION, current_state):
            return current_state

        # Transforme l'état actuel en tuple pour le suivi des explorations
        current_tuple = tuple(map(tuple, current_state))
        
        # Si déjà exploré, passe à l'état suivant
        if current_tuple in explored:
            continue

        # Marque l'état comme exploré
        explored.add(current_tuple)

        # Explore les actions possibles
        for action in available_actions(current_state):
            new_state = do_action(current_state, action)
            frontier.append(new_state)

    # Si aucune solution n'est trouvée
    return None

In [49]:
final_state = n_puzzle_with_explored(state)
ic(final_state)

ic| final_state: array([[1, 2, 3],
                        [4, 5, 6],
                        [7, 8, 0]])


array([[1, 2, 3],
       [4, 5, 6],
       [7, 8, 0]])